In [ ]:
import pandas as pd
import numpy
import sqlite3
import pyTigerGraph as tg

In [ ]:
# host = "http://medsales.34.106.148.222.nip.io:14240"
host = "http://protomolecule.magichome"
username = "tigergraph"
password = "Tigergraph"
graphName = "Northwind"
restppPort = 30900
gsPort = 30240 
mysecret = "254irou1ff6018fcoffd0qo38r8gofru"
# First establish a basic connection using a secret.  Do *not* do this if you already have a token
# host = "http://34.106.148.222.nip.io"
# token="o4luvshk47rticuhqi0cdokiq739o8i9"
conn = tg.TigerGraphConnection(host=host, restppPort=restppPort, gsPort=gsPort, graphname=graphName, password=password)
token = conn.getToken(mysecret, setToken=True, lifetime=None)

# Next use the new token to establish a full access connection for use with GSQL

conn = tg.TigerGraphConnection(host=host, restppPort=restppPort, gsPort=gsPort, graphname=graphName, password=password, apiToken=token[0])


In [ ]:
token[0]

In [ ]:
conn.getVertexTypes()

In [ ]:
# conn.getVertexTypes()
# conn.runInstalledQuery('floors')
print(conn.gsql('show edge *', options=[]))

In [ ]:
conn.gsql('drop graph gort', options=[])

In [ ]:
print(conn.gsql('show graph *', options=[]))

In [ ]:
orders = pd.read_csv("../graph/tigergraph/solutions/northwind/data/orders.csv")
order_details = pd.read_csv("../graph/tigergraph/solutions/northwind/data/order-details.csv")
customers = pd.read_csv("../graph/tigergraph/solutions/northwind/data/customers.csv")
reps = pd.read_csv("../graph/tigergraph/solutions/northwind/data/employees.csv")

In [ ]:
order_details.dtypes


In [ ]:
order_details.describe().round()

In [ ]:
order_details.info()

In [ ]:
order_details.head(20)

In [ ]:
%%bash
# rm -fv orders.db

In [ ]:
orders.info()

In [ ]:
orders[orders['CommissionAmt'].notnull()]

In [ ]:
conn = sqlite3.connect('orders.db')
orders.to_sql('orders', conn, if_exists='replace')
order_details.to_sql('order_details', conn, if_exists='replace')
conn.close()

In [ ]:
reps['BirthDate']

# Reps and customers